# Open AI Embeddings

In [10]:
import openai
from dotenv import load_dotenv
import os
load_dotenv()  # take environment variables from .env.

OPENKEY_API = os.getenv("OPENAI_KEY")
ORGANIZATION_ID = os.getenv("ORGANIZATION_ID")
openai.organization = ORGANIZATION_ID
# get this from top-right dropdown on OpenAI under organization > settings
openai.api_key = OPENKEY_API
# get API key from top-right dropdown on OpenAI website

openai.Engine.list()  # check we have authenticated

<OpenAIObject list at 0x7f34798b6410> JSON: {
  "data": [
    {
      "created": null,
      "id": "babbage",
      "object": "engine",
      "owner": "openai",
      "permissions": null,
      "ready": true
    },
    {
      "created": null,
      "id": "ada",
      "object": "engine",
      "owner": "openai",
      "permissions": null,
      "ready": true
    },
    {
      "created": null,
      "id": "davinci",
      "object": "engine",
      "owner": "openai",
      "permissions": null,
      "ready": true
    },
    {
      "created": null,
      "id": "text-embedding-ada-002",
      "object": "engine",
      "owner": "openai-internal",
      "permissions": null,
      "ready": true
    },
    {
      "created": null,
      "id": "babbage-code-search-code",
      "object": "engine",
      "owner": "openai-dev",
      "permissions": null,
      "ready": true
    },
    {
      "created": null,
      "id": "text-similarity-babbage-001",
      "object": "engine",
      "owner": "op

In [11]:
MODEL = "text-similarity-babbage-001"

res = openai.Embedding.create(
    input=[
        "Sample document text goes here",
        "there will be several phrases in each batch"
    ], engine=MODEL
)

In [12]:
# extract embeddings to a list
embeds = [record['embedding'] for record in res['data']]

In [19]:
print( len(embeds[0]    ) )

2048


# Populate the Index

## Load the Dataset

In [41]:
from datasets import load_dataset
from rich import print

faq_dataset = load_dataset("csv" , data_files="/home/null/code/utd_chatbot/FAQ on UTD Student Accesebility - Sheet1.csv")
print(faq_dataset)
## remove all the None values
faq_dataset = faq_dataset.filter(lambda x: x['Question'] is not None and x['Answering'] is not None)

Using custom data configuration default-b5279144beba40c5
Found cached dataset csv (/home/null/.cache/huggingface/datasets/csv/default-b5279144beba40c5/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)
100%|██████████| 1/1 [00:00<00:00, 371.84it/s]


DatasetDict({
    train: Dataset({
        features: ['Question', 'Answering', 'URL', 'Label'],
        num_rows: 23
    })
})

Loading cached processed dataset at /home/null/.cache/huggingface/datasets/csv/default-b5279144beba40c5/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-d8b863d6763fbde1.arrow


In [42]:
print(len(faq_dataset['train']))

20

## Init the pinecon client

In [1]:
import pinecone
PINECONE_APIKEY = os.getenv("PINECONE_APIKEY")
# initialize connection to pinecone (get API key at app.pinecone.io)
pinecone.init(
    api_key = "3bfba84d-b7a4-43f2-8931-c61ddddd69b1",
    environment="us-east1-gcp"
)

2141478c-ff8b-4881-b0b9-b78aa5bed0b9


In [44]:
index = pinecone.Index('elements')

# Populating the Index

In [45]:
from tqdm.auto import tqdm  # this is our progress bar

batch_size = 12  # process everything in batches of 32
for i in tqdm(range(0, len(faq_dataset['train']), batch_size)):
    # set end position of batch
    i_end = min(i+batch_size, len(faq_dataset['train']))
    # get batch of lines and IDs
    lines_batch = faq_dataset["train"]['Question'][i: i+batch_size]
    ids_batch = [str(n) for n in range(i, i_end)]
    # create embeddings
    print(lines_batch)
    res = openai.Embedding.create(input=lines_batch, engine=MODEL)
    embeds = [record['embedding'] for record in res['data']]
    # prep metadata and upsert batch
    meta = [{'text': line} for line in lines_batch]
    to_upsert = zip(ids_batch, embeds, meta)
    # upsert to Pinecone
    index.upsert(vectors=list(to_upsert))


  0%|          | 0/2 [00:00<?, ?it/s]

[
    'Does UT Dallas provide services for students with disabilities?',
    'Is there a separate admissions procedure for students with disabilities?\n',
    'Should I send my disability documentation with my admissions application?',
    'What documentation is required to receive disability services?\n',
    'Is there a deadline for submitting disability documentation?',
    'Who at the University will know about my disability if I register with ARC?',
    'What happens after my disability documentation is received?',
    'What accommodations and services will I be eligible for?',
    'Services and Procedures for Employees',
    'How do I take a test at the ARC Testing Center?\n',
    'I’ve taken exams at the ARC before, but now “Eligible to test in the ARC Testing Center” does not appear on my
accommodation letter. I want to test at the ARC again. How do I appeal this?',
    'Do I have to submit the form for every test, exam or quiz I want to take with you?\n'
]

 50%|█████     | 1/2 [00:00<00:00,  1.28it/s]

[
    'My professors said they would send the form in for me or schedule my exams at the ARC. This is OK, right?',
    'I want to change my reservation or the date of the exam or make a change to a reservation I already submitted.
Can I do that?',
    'Why can’t I start my exam on the half hour (e.g., 1:30 p.m., 2:30 p.m., 3:30 p.m.)?',
    'My professor says they cannot find the space or time for me to take my test with accommodations. Guess I will 
just take it with the class and hope for the best?',
    'Can I just ask the ARC to get the time or other details from my professor when I fill out the online test 
form?',
    'My professor gives a pop quiz or short test during part of the class period. How does this work with my 
accommodations?\n',
    'I’ve never been asked to do this. Why am I just now seeing a request like this? ',
    'Why can’t this student just show up to take tests? \n'
]

100%|██████████| 2/2 [00:01<00:00,  1.75it/s]


In [47]:
query = "I have ADHD. What do I do?"

xq = openai.Embedding.create(input=query, engine=MODEL)['data'][0]['embedding']
res = index.query([xq], top_k=5, include_metadata=True)
print(res)

{'matches': [{'id': '6',
              'metadata': {'text': 'What happens after my disability '
                                   'documentation is received?'},
              'score': 0.71944952,
              'sparseValues': {},
              'values': []},
             {'id': '17',
              'metadata': {'text': 'My professor gives a pop quiz or short '
                                   'test during part of the class period. How '
                                   'does this work with my accommodations?\n'},
              'score': 0.719309688,
              'sparseValues': {},
              'values': []},
             {'id': '7',
              'metadata': {'text': 'What accommodations and services will I be '
                                   'eligible for?'},
              'score': 0.713008881,
              'sparseValues': {},
              'values': []},
             {'id': '3',
              'metadata': {'text': 'What documentation is required to receive '
                                   'disability services?\n'},
              'score': 0.705391407,
              'sparseValues': {},
              'values': []},
             {'id': '19',
              'metadata': {'text': 'Why can’t this student just show up to '
                                   'take tests? \n'},
              'score': 0.703758121,
              'sparseValues': {},
              'values': []}],
 'namespace': ''}

In [63]:
from rich import print
print( res["matches"][0]["metadata"]["text"] )
template = """
Please act as a University of Texas Counselor. I will provide you with an individual 
looking for guidance at the University of Texas at Dallas, and your task is to help them 
solve their problem\n
"""
for query_response in  res["matches"]:
    answer = faq_dataset['train'].filter(lambda x: x['Question'] == query_response['metadata']['text'])['Answering'][0]
    template += f"Q: {query_response['metadata']['text']}\nA: {answer}\n"
prompt = template + f"Q: {query}\nA: "

What happens after my disability documentation is received?

100%|██████████| 1/1 [00:00<00:00, 192.80ba/s]


In [66]:
print(prompt)

Please act as a University of Texas Counselor. I will provide you with an individual 
looking for guidance at the University of Texas at Dallas, and your task is to help them 
solve their problem

Q: What happens after my disability documentation is received?
A: ARC evaluates documentation to determine eligibility for services. An intake appointment is set to identify 
reasonable accommodations, and an individualized service plan is created to meet the needs of the registered 
student. 

Please note academic accommodations are not retroactive.
Q: My professor gives a pop quiz or short test during part of the class period. How does this work with my 
accommodations?

A: Quizzes can be complicated to accommodate, especially if you do not know about them in advance. It’s important 
to work out the details early in the semester when you see on your syllabus that there will be quizzes in addition 
to regular exams. 

Many students decide not to use their accommodations on quizzes since they are typically short, and students would 
have to leave class to go to their eligible testing center. However, it is your choice, and if you do need to use 
your accommodations, they should be made available to you. 

If you decide you want to use your testing accommodations for quizzes and you know about them in advance, ask the 
professor if you can take the quiz earlier or later than the class period in a location they provide. If you are 
eligible to test at the ARC, then you may submit the appropriate test scheduling form. 

If you need a reader, scribe, digital text or any other accommodations that require significant lead time, it is 
extremely important to discuss these accommodations with the ARC and your professors as soon as possible for 
quizzes especially pop quizzes. 

The quizzes often are given after class starts; for example, if the class starts at 10 a.m., the test might begin 
at 10:30 a.m. If you are eligible to take the test at the ARC Testing Center, you are required to start all tests, 
exams or quizzes on the hour (e.g., 9 a.m., 10 a.m., etc.). In this situation, you will need to get clearance from 
the professor to take the quiz at any time the ARC Testing Center is open the same date the class takes the quiz or
test. 

If you schedule the test earlier or later than the class with the ARC and the professor approves, you will need to 
begin the test promptly at the start time you reserved. In other words, you cannot schedule a test at 10 a.m. in 
the ARC then show up at 10:30 a.m. to take the quiz when the rest of the class starts. If you are more than 15 
minutes late for your scheduled testing time, you will not be allowed to test. 

Some professors like to discuss the questions on the quiz during class when students are finished. If you arrange 
to take the quiz after the class due to an accommodation, you must leave the classroom when such discussions occur.

Faculty are sometimes using quizzes to check attendance (possibly by using clickers). If this is the case, contact 
the ARC as early as possible so we can work with you and the instructor to identify accommodation options.
Q: What accommodations and services will I be eligible for?
A: Appropriate and reasonable accommodations are determined on an individual basis. These accommodations may differ
from those provided in high school or other colleges. Accommodations frequently used by students with disabilities 
include extended time for test taking, note takers, printed materials in alternate formats, sign language 
interpreters and the use of assistive technology.
Q: What documentation is required to receive disability services?

A: Students requesting services must provide current and comprehensive documentation from a qualified professional.
The documentation must verify the disability and its impact on a student’s academic performance and functional 
limitations. 

Please refer to the Required Documentation Guidelines for complete details. Please note that eligi

In [64]:
response = openai.Completion.create(
  model="text-davinci-002",
  prompt =  prompt,
  temperature=0.3,
  max_tokens=150,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

In [65]:
print(response["choices"][0]["text"])

There are a few things you can do if you have ADHD and you want to succeed in college. First, you can get in touch 
with the Office of Disability Services (ODS) and request accommodations. To be eligible for accommodations, you 
will need to provide documentation from a qualified professional that verifies your diagnosis and explains how your
ADHD impacts your academic performance. 

Once you have been approved for accommodations, you will need to work with your professors to implement them. This 
may involve meeting with your professors before or after class, or sending them an email explaining your 
accommodations and how they should be implemented. 

In addition to accommodations, there are a few other things you can do to help you succeed in college. First,